In [2]:
# !pip3 freeze

aiofiles==23.2.1
aiohttp==3.9.3
aiosignal==1.3.1
altair==5.3.0
annotated-types==0.6.0
anyio==3.7.1
appnope==0.1.3
asttokens==2.2.1
async-timeout==4.0.3
asyncer==0.0.2
attrs==23.2.0
backcall==0.2.0
bidict==0.23.1
blinker==1.7.0
build==1.0.3
CacheControl==0.13.1
cachetools==5.3.3
certifi==2023.7.22
cffi==1.16.0
chainlit==1.0.301
charset-normalizer==3.3.2
cleo==2.1.0
click==8.1.7
comm==0.1.3
contourpy==1.0.7
crashtest==0.4.1
cycler==0.11.0
dataclasses-json==0.5.14
debugpy==1.6.7
decorator==5.1.1
Deprecated==1.2.14
distlib==0.3.8
distro==1.9.0
dnspython==2.4.2
dulwich==0.21.7
exceptiongroup==1.2.0
executing==1.2.0
fastapi==0.108.0
fastapi-socketio==0.0.10
fastjsonschema==2.19.1
filelock==3.13.1
filetype==1.2.0
fonttools==4.39.3
frozenlist==1.4.1
fsspec==2023.10.0
gitdb==4.0.11
GitPython==3.1.43
googleapis-common-protos==1.62.0
grpcio==1.64.0
grpcio-tools==1.62.2
h11==0.14.0
h2==4.1.0
hpack==4.0.0
httpcore==1.0.4
httpx==0.27.0
huggingface-hub==0.17.3
hyperframe==6.0.1
idna==3.4
importlib-me

In [2]:
# !python -m pip install "pymongo[srv]"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 KB 16.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 KB 31.6 MB/s eta 0:00:00


In [29]:
# !pip install pyopenssl --upgrade

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 4.4 MB/s eta 0:00:00


In [2]:
from utils.MongoDBChatMessageHistory  import MongoDBChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

In [4]:
import urllib 
MONGODB_URI = "mongodb://localhost:27017"
DB_NAME = "ARH_chatbot"

In [10]:
from pymongo import MongoClient
mclient = MongoClient(MONGODB_URI)
database = mclient["ARH_chatbot"]
database.list_collection_names()
collection = database["chat_history"]


In [12]:
collection.count_documents({})
cursor = collection.find({})
for document in cursor:
          print(document)
mclient.close()

{'_id': ObjectId('66787b9dcbd3da2cb01472b5'), 'SessionId': 'c8d4b195-971c-48a5-8f1c-b249acdc53ba', 'UserId': 'admin', 'History': '{"type": "human", "data": {"content": "What is BPM. Explain in 1 sentence", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}'}
{'_id': ObjectId('66787b9dcbd3da2cb01472b6'), 'SessionId': 'c8d4b195-971c-48a5-8f1c-b249acdc53ba', 'UserId': 'admin', 'History': '{"type": "ai", "data": {"content": "Business Process Management (BPM) refers to the systematic management and optimization of business processes, enabling organizations to improve efficiency, reduce costs, and enhance customer satisfaction through process automation, monitoring, and continuous improvement.", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": []}}'}
{'_id': ObjectId('66787f86cbd3da2cb01472b8'), 'SessionId': 'c8d4b195-971c-48a5-8f1c-b

In [7]:
import json
user_id = 'admin'

# MongoDB aggregation pipeline
pipeline = [
    {
        '$match': {
            'UserId': user_id  # Filter documents by UserId
        }
    },
    {
        '$group': {
            '_id': '$SessionId',  # Group by SessionId
            'user_id': {'$first': '$UserId'},  # Example: Get the first UserId in each group
            'count': {'$sum': 1},  # Example: Count documents in each group
            'History': {'$push': '$History'}  # Example: Push History field into an array
        }
    },
    {
        '$sort': {'_id': 1}  # Optional: Sort by SessionId (ascending)
    }
]

# Execute the aggregation pipeline
results = collection.aggregate(pipeline)
# for result in results:
#     print(result)

if results:
    items = [{"History": document["History"], "_id": document["_id"], "user_id": document["user_id"]} for document in results]
else:
    items = []

print(items)

for chat_index, chat in enumerate(items, start=1):
    chat_label = f"Chat {chat_index}"
    parsed_messages = [json.loads(message) for message in chat["History"]]
        # Display each message in an individual expander
    print(chat["_id"])
    for message_index, message in enumerate(parsed_messages, start=1):
            message_label = f"{message['type'].capitalize()} {message_index}: {message['data']['content'][:50]}..."
            print(message['data']['content'])

# parsed_items = [json.loads(item) for item in items[0]]

# Use Streamlit's expander to display each item
# for i, item in enumerate(parsed_items, start=1):
#    with st.expander(f"{item['type'].capitalize()} {i}: {item['data']['content'][:50]}..."):
#         st.write(item['data']['content'])


[{'History': ['{"type": "human", "data": {"content": "What is a Docker container? Explain in 1 sentence", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}', '{"type": "ai", "data": {"content": "A Docker container is a lightweight and portable software package that includes an application, its dependencies, and the necessary runtime environment to run it, all packaged together within a single file.", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": []}}'], '_id': '0e796a7c-42ff-4277-b982-383b9987eb20', 'user_id': 'admin'}, {'History': ['{"type": "human", "data": {"content": "What is BPM. Explain in 1 sentence", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}', '{"type": "ai", "data": {"content": "Business Process Management (BPM) refers to the systematic management

In [9]:
collection.delete_many({})

DeleteResult({'n': 4, 'ok': 1.0}, acknowledged=True)

In [2]:
from dbutils.MongoDBChatMessageHistory import MongoDBChatMessageHistory

def get_session_history(session_id: str,user_id: str) -> MongoDBChatMessageHistory:
        return MongoDBChatMessageHistory(MONGODB_URI, session_id,user_id, database_name=DB_NAME, collection_name="chat_history")

In [7]:
get_session_history("", "").getformatedmessage(pipeline)

[{'type': 'human',
  'data': {'content': 'What is BPM. Explain in 1 sentence',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': 'Business Process Management (BPM) refers to the systematic management and optimization of business processes, enabling organizations to improve efficiency, reduce costs, and enhance customer satisfaction through process automation, monitoring, and continuous improvement.',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'ai',
   'name': None,
   'id': None,
   'example': False,
   'tool_calls': [],
   'invalid_tool_calls': []}},
 {'type': 'human',
  'data': {'content': 'What is an API. Explain in 1 sentence',
   'additional_kwargs': {},
   'response_metadata': {},
   'type': 'human',
   'name': None,
   'id': None,
   'example': False}},
 {'type': 'ai',
  'data': {'content': 'An Application Programming Interface (API) is a

In [4]:
from qdrant_client import QdrantClient
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
client = QdrantClient(host="129.69.217.24", port=6333)

In [5]:
embed_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
qdrant = Qdrant(client=client, collection_name="ARH_Tool", embeddings=embed_model)

In [7]:
retriever = qdrant.as_retriever()

In [8]:
inference_server_url = "http://129.69.217.24:8009/v1"

# create LLaamacpp Langchain instance
model = ChatOpenAI(
    model="no-path",
    openai_api_key="no-key",
    openai_api_base=inference_server_url,
    # max_tokens=5,
    temperature=0,
    streaming=True
)

In [ ]:
# def format_docs_with_id(docs) -> str:
#     formatted = [
#         f"Source ID: {i}\nArticle Title: {doc.metadata['source']}\nArticle Snippet: {doc.page_content}"
#         for i, doc in enumerate(docs)
#     ]
#     return "\n\n" + "\n\n".join(formatted)

In [9]:
# Given a follow-up question and history, create a standalone question
standalone_system_prompt = """
Given a chat history and a follow-up question, rephrase the follow-up question to be a standalone question. \
Do NOT answer the question, just reformulate it if needed, otherwise return it as is. \
Only return the final standalone question. \
"""
standalone_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", standalone_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
parse_output = StrOutputParser()

question_chain =  standalone_question_prompt | model  | parse_output 

In [93]:
# retriever_chain = (lambda x: x["question"]) | retriever

In [10]:
# Generate context by passing output of the question_chain i.e. the standalone question to the retriever
retriever_chain = RunnablePassthrough.assign(context=question_chain | retriever | (lambda docs: {
        'content': "\n\n".join([d.page_content for d in docs]),
        'sources': [d.metadata['source'] for d in docs]
    }))

In [13]:
# Create a prompt that includes the context, history and the follow-up question
rag_system_prompt = """Answer the question based only on the following context: \
{context}
"""
rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rag_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

In [18]:
# Define a global variable to hold the output of retriever_chain
retriever_output = None

# Define a function to capture the output of retriever_chain
def capture_output(output):
    global retriever_output
    retriever_output = output
    return output

# RAG chain
# rag_chain = (
#     retriever_chain
#     | RunnablePassthrough.assign(prompt=rag_prompt)
#     | RunnablePassthrough.assign(response=lambda inputs: model(inputs["prompt"].messages))
#     | RunnablePassthrough.assign(what=parse_output)
# )

rag_chain = (
    retriever_chain
    | capture_output 
    | rag_prompt
    | model
    | parse_output
)

In [95]:
# new_chain = RunnablePassthrough.assign(context=retriever_chain).assign(
#     answer=rag_chain
# )

In [19]:
# RAG chain with history
with_message_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",
    output_messages_key="answer",
    history_messages_key="chat_history",
)

In [38]:
%%time
with_message_history.stream({"question": "How is the microservice deployed into the production environment?"}, {"configurable": {"session_id": "1"}})

CPU times: user 9 µs, sys: 6 µs, total: 15 µs
Wall time: 25 µs


<generator object RunnableBindingBase.stream at 0x71f0ea654c10>

In [75]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017") 


In [85]:
database = client.get_database('ARH_chatbot') 
collection_list = database.list_collections()
for c in collection_list:
    print(c)

{'name': 'chat_history', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\x8d\x1e\x7f:!\xc9K\x95\xad\xc2\xfbNf\x8d%j', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}


In [56]:
ans = with_message_history.invoke(
    {"question": "What is BPM?"},
    config={"configurable": {"session_id": "abc123","user": "user1"}},
)
ans

Parent run e22cddf8-89f7-4ebb-b25e-5e680bbbe757 not found for run abc52ebc-8016-489f-a6c1-4142173442c8. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: KeyError('answer')


'BPM stands for Business Process Management. It refers to the management of business processes, which are sets of activities that create value for an organization by transforming inputs into outputs.\n\nBusiness process management involves designing, modeling, executing, and monitoring business processes to achieve specific goals and objectives. BPM aims to improve the efficiency, effectiveness, and agility of business processes by:\n\n1. Identifying and documenting existing processes\n2. Analyzing and optimizing processes to eliminate waste and inefficiencies\n3. Automating repetitive tasks and manual processes\n4. Implementing controls and monitoring systems to ensure compliance and quality\n5. Continuously improving and refining processes based on feedback and performance metrics\n\nBPM is used in various industries, including finance, healthcare, manufacturing, logistics, and government, among others. It helps organizations to:\n\n* Increase productivity and efficiency\n* Reduce co

In [45]:
# ans.keys()

retriever_output["context"]["sources"]

AttributeError: 'dict' object has no attribute 'sources'

In [ ]:
ct = 0
chat_history=[]
for jsonpatch_op in with_message_history.stream(
    {"question": "How is the microservice deployed into the production environment?"}, {"configurable": {"session_id": "1"}}
):
    print(jsonpatch_op)
    # print("\n" + "-" * 30 + "\n")
    # ct += 1
    # if ct > 20:
    #     break

In [84]:
get_session_history("abc123").messages

[HumanMessage(content='When was he elected?'),
 AIMessage(content="I apologize, but there is no information about an election in the provided context. The text appears to be a passage from a research paper or article about software engineering and microservices architecture. It does not mention any individual being elected. If you could provide more context or clarify what you are referring to, I'll do my best to help."),
 HumanMessage(content='When was he elected?'),
 AIMessage(content="I apologize again, but there is no information about an election or a person being elected in the provided context. The text appears to be a passage from a research paper or article about software engineering and microservices architecture, and does not mention any individual being elected. If you could provide more context or clarify what you are referring to, I'll do my best to help.")]

In [41]:
import os
os.environ["LANGCHAIN_API_KEY"] = "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"  
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "callback-experiments" 

In [57]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
from torch import cuda
import os
cuda.current_device()

0

In [16]:
cuda.set_device(1)

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [24]:
!python3 -m llama_cpp.server --model /tmp/llama_index/models/Meta-Llama-3-8B-Instruct.Q5_K_M.gguf --port 8009 --host 129.69.217.24 --n_ctx 8000 --chat_format chatml

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /tmp/llama_index/models/Meta-Llama-3-8B-Instruct.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 llama.ro

In [34]:
import streamlit as st

In [35]:
# Page title
st.set_page_config(page_title='Architecture Refactoring AI Helper')
st.title('Architecture Refactoring AI Helper')

2024-05-24 18:17:02.508 
  command:

    streamlit run /usr/lib/python3/dist-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [16]:
!nvidia-smi

Sun Jun  9 14:12:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        On  | 00000000:01:00.0 Off |                  Off |
|  0%   36C    P8              20W / 450W |  22861MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import psycopg2

ModuleNotFoundError: No module named 'psycopg2'